In [ ]:
import sqlite3
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from typing import Annotated, Literal, Sequence
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import AIMessage
from typing_extensions import TypedDict
from langgraph.graph.message import AnyMessage, add_messages
from typing import Any
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda, RunnableWithFallbacks
from langgraph.prebuilt import ToolNode
from langgraph.graph import END, StateGraph, START
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod


In [ ]:
load_dotenv()

In [ ]:
connection = sqlite3.connect("employee.db")

In [ ]:
connection

In [ ]:
table_creation_query="""
CREATE TABLE IF NOT EXISTS employees (
emp_id INTEGER PRIMARY KEY,
first_name TEXT NOT NULL,
last_name TEXT NOT NULL,
email TEXT UNIQUE NOT NULL,
hire_date TEXT NOT NULL,
salary REAL NOT NULL

);
"""

In [ ]:
table_creation_query2="""
CREATE TABLE IF NOT EXISTS customers (
customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
first_name TEXT NOT NULL,
last_name TEXT NOT NULL,
email TEXT UNIQUE NOT NULL,
phone TEXT
);
"""

In [ ]:
table_creation_query3="""
CREATE TABLE IF NOT EXISTS orders (
order_id INTEGER PRIMARY KEY AUTOINCREMENT,
customer_id INTEGER NOT NULL,
order_date TEXT NOT NULL,
amount REAL NOT NULL,
FOREIGN KEY (customer_id) REFERENCES customers (customer_id)

);
"""

In [ ]:
cursor = connection.cursor()

In [ ]:
cursor.execute(table_creation_query)

In [ ]:
cursor.execute(table_creation_query2)

In [ ]:
cursor.execute(table_creation_query3)

In [ ]:
insert_query = """
INSERT INTO employees (emp_id, first_name, last_name, email, hire_date, salary)
VALUES (?, ?, ?, ?, ?, ?);
"""

In [ ]:
insert_query_customers = """
INSERT INTO customers (customer_id, first_name, last_name, email, phone)
VALUES (?, ?, ?, ?, ?);
"""

In [ ]:
insert_query_orders = """
INSERT INTO orders (order_id, customer_id, order_date, amount)
VALUES (?, ?, ?, ?);
"""

In [ ]:
employee_data = [
(1, "Sunny", "Savita", "sunny.sv@abc.com", "2023-06-01", 50000.00),
(2, "Hamid", "Gholami", "hamid.megmail.com", "2022-04-15", 6000.00),
(3, "Alice", "Johnson", "alice.johnson@jpg.com", "2021-09-30", 55000.00),
(4, "Bob", "Brown", "bob.brown@uio.com", "2020-01-20", 45000.00),
]

In [ ]:
customers_data = [
(1, "John", "Doe", "john.doe@example.com", "1234567890"),
(2, "Jane", "Smith", "jane.smith@example.com", "9876543210"),
(3, "Emily", "Davis", "emily.davis@example.com", "4567891230"),
(4, "Michael", "Brown", "michael.brown@example.com", "7894561230"),
]

In [ ]:
orders_data = [
(1, 1, "2023-12-01", 250.75),
(2, 2, "2023-11-20", 150.50),
(3, 3, "2023-11-25", 300.00),
(4, 4, "2023-12-02", 450.00),
]

In [ ]:
cursor.executemany(insert_query, employee_data)

In [ ]:
cursor.executemany(insert_query_customers, customers_data)

In [ ]:
cursor.executemany(insert_query_orders, orders_data)

In [ ]:
connection.commit()

In [ ]:
cursor.execute("select * from employees")

In [ ]:
for row in cursor.fetchall():
    print(row)

In [ ]:
cursor.execute("select first_name from employees where salary > 50000.0;")

In [ ]:
cursor.fetchall()

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type = 'table';")

In [ ]:
cursor.fetchall()

In [ ]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [ ]:
llm = ChatGroq(model="llama3-70b-8192")

In [ ]:
llm.invoke("hi").content

In [ ]:
db = SQLDatabase.from_uri("sqlite:///employee.db")

In [ ]:
print("Dialect: ", db.dialect)
print("Usable tables: ", db.get_usable_table_names())

In [ ]:
query_result = db.run("SELECT * FROM employees ; ")
print("Query result from Employees table: \n", query_result)

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [ ]:
tools = toolkit.get_tools()

In [ ]:
for tool in tools:
    print(tool.name)

In [ ]:
list_tables_tool = next((tool for tool in tools if tool.name == "sql_db_list_tables"), None)

In [ ]:
list_tables_tool

In [ ]:
get_schema_tool = next((tool for tool in tools if tool.name == "sql_db_schema"), None)

In [ ]:
get_schema_tool

In [ ]:
print(list_tables_tool.invoke(""))

In [ ]:
print(get_schema_tool.invoke("customers"))

In [ ]:
@tool
def db_query_tool(query:str) -> str:
    """
    Execute a SQL query against the database and return the result.
    If the query is invalid or returns no result, an error message will be returned.
    In case of an error, the user is advised to rewrite the query and try again.
    """

    result = db.run_no_throw(query)
    if not result:
        return "Error: Query failed. Please rewrite your query and try again."
    
    return result

In [ ]:
print(db_query_tool.invoke("SELECT * FROM Employees LIMIT 10; "))

In [ ]:
class Database:
    def run_no_throw(self, query):
        try:
            # Assume 'self.connection' is a valid database connection
            cursor = self.connection.cursor()
            cursor.execute(query)
            return cursor. fetchall() # Or another method to retrieve results
        except Exception as e:
            print(f"Error executing query: {e}")
            return None # Fallback value

In [ ]:
class SubmitFinalAnswer(BaseModel):
    """ Submit the final answer to the user based on the query results. """
    final_answer : str = Field(..., description="The final answer to user")

In [ ]:
class State(TypedDict):
    message : Annotated[list[AnyMessage], add_messages]

In [ ]:
query_check_system = """You are a SQL expert with a strong attention to detail.
Double check the SQLite query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.

You will call the appropriate tool to execute the query after running this check."""

query_check_prompt = ChatPromptTemplate.from_messages([("system", query_check_system), ("placeholder", "{messages}")])
query_check = query_check_prompt | llm.bind_tools([db_query_tool])

query_check.invoke({"messages": [("user", "SELECT * FROM Employees LIMIT 5;")]})

In [ ]:
# Add a node for a model to generate a query based on the question and schema
query_gen_system = """You are a SQL expert with a strong attention to detail.

Given an input question, output a syntactically correct SQLite query to run, then look at the results of the query and return the answer.

DO NOT call any tool besides SubmitFinalAnswer to submit the final answer.

When generating the query:

Output the SQL query that answers the input question without a tool call.

Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.

If you get an error while executing a query, rewrite the query and try again.

If you get an empty result set, you should try to rewrite the query to get a non-empty result set.
NEVER make stuff up if you don't have enough information to answer the query ... just say you don't have enough information.

If you have enough information to answer the input question, simply invoke the appropriate tool to submit the final answer to the user.

DO NOT make any DML statements (INSERT. UPDATE. DELETE, DROP etc.) to the database. Do not return any sql query except answer. """

query_gen_prompt = ChatPromptTemplate.from_messages([("system", query_gen_system), ("placeholder", "{messages}")])

query_gen = query_gen_prompt | llm.bind_tools([SubmitFinalAnswer])

In [ ]:
def first_tool_call(state: State) -> dict[str, list[AIMessage]]:
    return {"messages": [AIMessage(content="", tool_calls=[{"name": "sql_db_list_tables", "args":{}, "id": "tool_abcd123"}])]}

In [ ]:
def handle_tool_error(state:State) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls

    return {
        "messages":[
            ToolMessage(content=f"Error: {repr(error)}\n please fix your mistakes. ", tool_calls_id=tc["id"],)
            for tc in tool_calls
        ]
    }

def create_tool_node_with_fallback(tools:list) -> RunnableWithFallbacks[Any, dict]:
    return ToolNode(tools).with_fallbacks([RunnableLambda(handle_tool_error)], exception_key="error")

In [ ]:
def query_gen_node(state:State):
    message = query_gen.invoke(state)

    # Sometimes, the LLM will hallucinate and call the wrong tool, we need to catch this and return an error message.
    tool_messages = []
    if message.tool_calls:
        for tc in message.tool_calls:
            if tc["name"] != "SubmitFinalAnswer":
                tool_messages.append(
                    ToolMessage(
                        content=f"Error: The wrong tool was called: {tc["name"]}. Please fix your mistakes. Remember to only call SubmitFinalAnswer.",
                        tool_call_id=["id"],
                    )
                )

            else:
                tool_messages = []

            return {"messages": [message] + tool_messages}

In [ ]:
def should_continue(state: State) -> Literal[END, "Correct_query", "query_gen"]:
    messages = state["messages"]
    last_message = messages[-1]

    if getattr(last_message, "tool_calls", None):
        return END
    
    if last_message.content.startswith("Error:"):
        return "query_gen"
    
    else:
        return "Correct_query"

In [ ]:
def model_check_query(state: State) -> dict[str, list[AnyMessage]]:
    """
    Use this tool to double-check if your query is corret before executing it.
    """
    return {"messages": [query_check.invoke({"messages": [state["messages"][-1]]})]}

In [ ]:
workflow = StateGraph(State)

workflow.add_node("first_tool_call", first_tool_call)
workflow.add_node("list_tables_tool", create_tool_node_with_fallback([list_tables_tool]))
workflow.add_node("get_schema_tool", create_tool_node_with_fallback([get_schema_tool]))
model_get_schema = llm.bind_tools([get_schema_tool])
workflow.add_node("model_get_schema", lambda state: {"messages": [model_get_schema.invoke(state["messages"])]})
workflow.add_node("query_gen", query_gen_node)
workflow.add_node("correct_query", model_check_query)
workflow.add_node("execute_query", create_tool_node_with_fallback([db_query_tool]))

In [ ]:
workflow.add_edge(START, "first_tool_call")
workflow.add_edge("first_tool_call", "list_tables_tool")
workflow.add_edge("list_tables_tool", "model_get_schema")
workflow.add_edge("model_get_schema", "get_schema_tool")
workflow.add_edge("get_schema_tool", "query_gen")
workflow.add_conditional_edges("query_gen", should_continue)
workflow.add_edge("correct_query", "execute_query")
workflow.add_edge("execute_query", "query_gen")

In [ ]:
app = workflow.compile()

In [ ]:
display(
    Image(
        app.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API
        )
    )
)

In [ ]:
query = {"messages": [("user", "Tell me name of the employee who's salary is more than 30000")]}

In [ ]:
response = app.invoke(query)

In [ ]:
response["messages"][-1].tool_calls[0]["args"]["final_answer"]

In [ ]:
query = {"messages": [("user", "Tell me about all the orders")]}

In [ ]:
response = app.invoke(query)

In [ ]:
response["messages"][-1].tool_calls[0]["args"]["final_answer"]